### Collecting Steam Games Data

In [ ]:
import ast
import json
import pandas as pd
import requests

from collections import defaultdict
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import warnings

warnings.filterwarnings('ignore')

with open('user.txt', 'r') as file:
	STEAM_ID, API_KEY = file.read().split('\n')

GAMES_URL = 'https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001?key={}'
GAMES_URL = GAMES_URL.format(API_KEY)
GAMES_URL = GAMES_URL + '&include_played_free_games=1&skip_unvetted_apps=0&include_appinfo=1&steamid={}'
GAMES_URL = GAMES_URL.format(STEAM_ID)

ACHIEV_URL = '&key=' + API_KEY + '&steamid=' + STEAM_ID
ACHIEV_URL = 'https://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid={}' + ACHIEV_URL

APP_URL = 'https://api.steamcmd.net/v1/info/{}'

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [ ]:
response = session.get(GAMES_URL)
result = response.json()["response"]["games"]
result = sorted(result, key=lambda g: g["playtime_forever"], reverse=True)
df = pd.json_normalize(result)

### Collecting Additional Data

In [ ]:
achievements_data = []

for appid, name in zip(df['appid'], df['name']):
	response = session.get(ACHIEV_URL.format(appid))

	try:
		result = response.json()['playerstats']['achievements']
		achievements = pd.json_normalize(result)
		unlocked = (achievements['achieved'] == 1).sum()
		locked = (achievements['achieved'] == 0).sum()
		total = unlocked + locked
		perfect = int(unlocked == total)

	except KeyError:
		unlocked, locked, total, perfect = 0, 0, 0, 0

	achievements_data.append([appid, unlocked, locked, total, perfect])
	print("Collected achievements for {}".format(name))

achievements_df = pd.DataFrame(achievements_data, columns=['appid', 'achievements_unlocked', 'achievements_locked', 'achievements_total', 'achievements_perfect'])
df = df.merge(achievements_df, on='appid')

In [ ]:
with open("steamdb_tags.json", "r", encoding="utf-8") as f:
	tags_dict = json.load(f)

for appid, name in zip(df['appid'], df['name']):
	response = session.get(APP_URL.format(appid))

	try:
		associations = response.json()['data'][str(appid)]['common']['associations']
		n = len(associations)
		developer, publisher, franchise = [], [], []

		for i in range(n):
			asso = associations[str(i)]
			match(asso['type']):
				case 'developer':
					developer.append(asso['name'])
				case 'publisher':
					publisher.append(asso['name'])
				case 'franchise':
					franchise.append(asso['name'])

		df.loc[df.appid == appid, 'developer'] = str(developer)
		df.loc[df.appid == appid, 'publisher'] = str(publisher)
		df.loc[df.appid == appid, 'franchise'] = str(franchise)

	except KeyError:
		print("No associations for {}".format(name))

	try:
		tags = response.json()['data'][str(appid)]['common']['store_tags']
		tags = dict(sorted(tags.items(), key=lambda x: int(x[0])))

		for n, tag in tags.items():
			tag = tags_dict[str(tag)]
			df.loc[df.appid == appid, 'tag_{}'.format(n)] = tag

	except KeyError:
		print("No tags for {}".format(name))

	print("Collected info for {}".format(name))

In [ ]:
df.to_csv('games.tsv', sep='\t', index=False)

### Calcuting Stats

In [ ]:
weights = [round(max(0, 1 - i * 0.0265), 2) for i in range(20)]

tag_playtime = defaultdict(float)
tag_playtime_weighted = defaultdict(float)

for _, row in df.iterrows():
	playtime = row['playtime_forever']
	for i in range(20):
		tag_column = f'tag_{i}'
		if pd.notna(row[tag_column]):
			tag = row[tag_column]
			tag_playtime[tag] += playtime
			tag_playtime_weighted[tag] += playtime * weights[i]

tag_playtime_df = pd.DataFrame(tag_playtime.items(), columns=["Tag", "Total Playtime"])
tag_playtime_df['Total Playtime (hours)'] = round(tag_playtime_df['Total Playtime'] / 60, 2)

tag_playtime_weighted_df = pd.DataFrame(tag_playtime_weighted.items(), columns=["Tag", "Total Weighted Playtime"])
tag_playtime_weighted_df['Total Weighted Playtime (hours)'] = round(tag_playtime_weighted_df['Total Weighted Playtime'] / 60, 2)
tag_playtime_weighted_df['Total Weighted Playtime'] = round(tag_playtime_weighted_df['Total Weighted Playtime'], 2)

tag_playtime_df = pd.merge(tag_playtime_df, tag_playtime_weighted_df, on="Tag")
tag_playtime_df = tag_playtime_df.sort_values(by="Total Weighted Playtime", ascending=False)

In [ ]:
developer_playtime = defaultdict(float)
developer_count = defaultdict(int)
publisher_playtime = defaultdict(float)
publisher_count = defaultdict(int)
franchise_playtime = defaultdict(float)
franchise_count = defaultdict(int)

for _, row in df.iterrows():
	playtime = row['playtime_forever']
	developer = ast.literal_eval(row['developer'])
	publisher = ast.literal_eval(row['publisher'])
	franchise = ast.literal_eval(row['franchise'])

	for dev in developer:
		developer_playtime[dev] += playtime
		developer_count[dev] += 1

	for pub in publisher:
		publisher_playtime[pub] += playtime
		publisher_count[pub] += 1

	for fran in franchise:
		franchise_playtime[fran] += playtime
		franchise_count[fran] += 1

developer_playtime_df = pd.DataFrame(developer_playtime.items(), columns=['Developer', 'Playtime'])
publisher_playtime_df = pd.DataFrame(publisher_playtime.items(), columns=['Publisher', 'Playtime'])
franchise_playtime_df = pd.DataFrame(franchise_playtime.items(), columns=['Franchise', 'Playtime'])

developer_playtime_df['Playtime (hours)'] = round(developer_playtime_df['Playtime'] / 60, 2)
developer_playtime_df['Playtime'] = round(developer_playtime_df['Playtime'], 2)
publisher_playtime_df['Playtime (hours)'] = round(publisher_playtime_df['Playtime'] / 60, 2)
publisher_playtime_df['Playtime'] = round(publisher_playtime_df['Playtime'], 2)
franchise_playtime_df['Playtime (hours)'] = round(franchise_playtime_df['Playtime'] / 60, 2)
franchise_playtime_df['Playtime'] = round(franchise_playtime_df['Playtime'], 2)

developer_count_df = pd.DataFrame(developer_count.items(), columns=['Developer', 'Count'])
publisher_count_df = pd.DataFrame(publisher_count.items(), columns=['Publisher', 'Count'])
franchise_count_df = pd.DataFrame(franchise_count.items(), columns=['Franchise', 'Count'])

developer_playtime_df = pd.merge(developer_playtime_df, developer_count_df, on='Developer', how='left')
publisher_playtime_df = pd.merge(publisher_playtime_df, publisher_count_df, on='Publisher', how='left')
franchise_playtime_df = pd.merge(franchise_playtime_df, franchise_count_df, on='Franchise', how='left')

developer_playtime_df = developer_playtime_df.sort_values(by='Playtime', ascending=False)
publisher_playtime_df = publisher_playtime_df.sort_values(by='Playtime', ascending=False)
franchise_playtime_df = franchise_playtime_df.sort_values(by='Playtime', ascending=False)